# 🦙 Text2SQL Llama Analyst Demo

👋🏼 Welcome to my interactive demo which showcases my fine-tuned `LLaMA-2-7B` model that translates natural language questions into SQL queries.

It’s a fun and practical example of how large language models can be adapted to domain-specific tasks using lightweight fine-tuning techniques like LoRA and quantization.

I really hope you enjoy my demo, learning about my project, and seeing the reflections I've made along the way!

## 💡 What This Project Does

- **Task**: Converts natural language questions like "What are the names of all customers who placed an order in 2025?" into structured SQL queries.
- **Model**: `LLaMA-2-7B`, fine-tuned using parameter-efficient fine-tuning techniques: [LoRA (Low-Rank Adaptation)](https://arxiv.org/abs/2106.09685) and Quantization (Explained in detail below.)
- **Dataset**: [ChrisHayduk/Llama-2-SQL-Dataset](https://huggingface.co/datasets/ChrisHayduk/Llama-2-SQL-Dataset), which is a curated dataset of natural language-to-SQL examples.

## 💭 Why This Matters

- **Large Language Models (LLMs)** are great at performing general tasks, but they aren't specialists.
  > Fine-tuning enables them to perform highly specific tasks with greater accuracy and consistency.
- My project demonstrates how to **align a foundation model with user intent** through **parameter-efficient fine-tuning**.
- You’ll see how a base LLaMA model can become a **task-specific assistant** for querying structured databases.
- Additionally, you'll be able to **compare my model's SQL predictions against ground truth** examples from the evaluation set, which is a great way to explore and see my model's strengths and limitations first-hand.



## 🔧 A Deeper Look into LoRA + Quantization

My project uses two modern techniques to make training large models like LLaMA-2 efficient and accessible, even on limited hardware.

### 👉🏻 LoRA (Low-Rank Adaptation)

- Instead of updating all 7 billion parameters in the base LLaMA model, **LoRA freezes the original weights** and only targets/retrains specific modules like the attention and feedforward layers of the model. We move away from full-fine-tuning and only focus our attention on updating a fraction of our parameters (those relevant to our task).
- This dramatically reduces memory and compute needs, while still allowing the model to **learn new tasks like Text-to-SQL**.

### 👉🏻 4-bit Quantization

- Normally, LLaMA-2 uses 16-bit or 32-bit weights which is memory-heavy.
- **Quantization shrinks the model weights to just 4 bits**, making them **smaller and faster to load**, while still retaining performance.
- This allows us to **run the model on limited GPUs or even CPUs** for inference.


Together, **LoRA + Quantization (QLoRA)** allow us to fine-tune and deploy a powerful language model using minimal resources (making cutting-edge AI more accessible and efficient).


## 👨‍💻 Try It Yourself

Follow the cells below to:
- Setup my demo code
- Enter your own natural language questions and get corresponding SQL outputs
- Explore sample examples from the evaluation set and compare my model’s SQL predictions to the correct/expected SQL outputs

In [ ]:
'''SETUP: Cloning my Repo and Loading Our Fine-Tuned Model'''

# Clone my Repo
!git clone https://github.com/Akhan521/Text2SQL-LLaMA-Analyst.git
%cd Text2SQL-LLaMA-Analyst

# Install Dependencies
!pip install datasets
!pip install transformers -U
!pip install accelerate -U
!pip install bitsandbytes
!pip install peft


In [2]:
# 1. Loading our Base Model (Necessary for Loading our Fine-Tuned Model)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Base Model (Llama-2-7b-hf from Hugging Face)
base_model_name = "NousResearch/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# 2. Loading our Fine-Tuned Model (On Top of our Base Model)
from peft import PeftModel

# Note: Our Fine-Tuned Model is a LoRA Adaptor.
finetuned_model_name = "akhan365/llama2-finetuned-for-text2sql"
finetuned_model = PeftModel.from_pretrained(model, finetuned_model_name)

# 3. Loading our Tokenizer.
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("✅ Fine-tuned model loaded and ready for inference!")


✅ Fine-tuned model loaded and ready for inference!


In [10]:
from src.generate import generate

def generate_query(prompt: str):

    print("\n🤖 Fine-Tuned Llama-2 Response:")
    print("=" * 50)
    finetuned_output = generate(prompt, model, tokenizer, device=device)

    print(finetuned_output)

In [16]:
# Play around with the prompt:

'''
We'll go ahead and compare our models using a sample question from our evaluation set.
We'll also provide the expected solution for the given sample question.
'''
from src.data_loader import load_data

_, eval_dataset = load_data(dataset_name = "ChrisHayduk/Llama-2-SQL-Dataset")

# If you modify this prompt, make sure to re-run this code cell!
chosen_sample = torch.randint(0, len(eval_dataset), ()).item() # A random sample question from our eval. dataset.
sample_question = eval_dataset[chosen_sample]['input']
correct_answer  = eval_dataset[chosen_sample]['output']

generate_query(sample_question)

print("\n🗝️ Expected Answer:")
print("=" * 50)
print(correct_answer)


🤖 Fine-Tuned Llama-2 Response:
Below is an instruction that describes a SQL generation task, paired with an input that provides further context about the available table schemas. Write SQL code that appropriately answers the request.

### Instruction:
Which TV Station has a Romaji Title of kegareta shita?

### Input:
CREATE TABLE table_name_43 (tv_station VARCHAR, romaji_title VARCHAR)

### Response:  SELECT tv station FROM Table Name WHERE romanji title = "kega retta ShITa"

🗝️ Expected Answer:
SELECT tv_station FROM table_name_43 WHERE romaji_title = "kegareta shita"


### 💡 Potential Improvements

While my fine-tuned LLaMA-2 model shows meaningful improvements over the base model on translating natural language to SQL, there is still much room for growth:

- 🔍 **Output Quality**: My model may produce incomplete or syntactically incorrect SQL queries, especially on complex questions.
- 📊 **Dataset Limitations**: The training dataset (`ChrisHayduk/Llama-2-SQL-Dataset`) is relatively small, which limits my model’s generalization ability.
- 📏 **Evaluation Metrics**: A more rigorous evaluation framework using string-match accuracy, execution accuracy, or SQL parse correctness would provide deeper insight into model performance.
- ❔ **Question Understanding**: My model sometimes misinterprets ambiguous or vague natural language questions; Improving my model's ability to understand intent could lead to more accurate SQL generation.


Despite these challenges, my model consistently **outperforms the base LLaMA-2** on this task, demonstrating how even lightweight fine-tuning with LoRA can yield domain-specific gains with minimal compute.


### ⌛ Final Remarks

My project demonstrates how even a massive open-source model like **LLaMA-2** can be adapted to a highly specific task like **Text-to-SQL Generation** with **low-rank adaptation (LoRA)** and **parameter-efficient fine-tuning**.

Through this project, I gained experience in:

- Fine-tuning large language models with **PEFT (LoRA)** techniques
- Managing **quantization trade-offs** for faster training/inference
- Designing clean, modular training and inference workflows
- Publishing models to the **Hugging Face Hub**
- Building a demo that's accessible to both technical and non-technical users

While I believe there’s still much room for improvement, this project shows the real-world viability of domain adaptation using open-source models/weights.


📁 **My Repository**: [Text2SQL-LLaMA-Analyst](https://github.com/Akhan521/Text2SQL-LLaMA-Analyst)  
🌐 **My Portfolio**: [https://aamir-khans-portfolio.vercel.app/](https://aamir-khans-portfolio.vercel.app/)  
🔗 **My LinkedIn**: [https://www.linkedin.com/in/aamir-khan-aak521/](https://www.linkedin.com/in/aamir-khan-aak521/)

Thanks for checking out my demo! Feel free to explore my code, play around with my model, or reach out for fun conversations!
